## 取得字元与索引的对应

In [53]:
chars = [' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '8', '9', ':', ';', '<', '>', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x85']
char_indices = dict((char, chars.index(char)) for char in chars)

## 读取模型

In [54]:
from keras.models import load_model
model = load_model("imdb.hdf5")

## 随机采样文字

### Softmax With Temperature

$$p_t(a)= \frac{e^\frac{q_t(a)}{T}}{\sum_{i=0}^n e^\frac{q_t(i)}{T}}$$

- T被称为是温度参数（temperature parameter）。当T很大时，所有的激活值对应的激活概率趋近于相同（激活概率差异性较小）；而当T很低时，即趋于0时，不同的激活值对应的激活概率差异也就越大
    - temperature 低 ：会出现重复率比较高但是部分结构比较真实的文章。
    - temperature 高 ：文章会比较有意义，但可能会发明一些新词，词的部分结构可能不完整，有的词只有单词的一半
    - temperature=0.5：文章生成得比较好，在结构和随机性保持平衡的情况下，产生比较好的句子。

In [57]:
import numpy as np
def softmax(x, t):
    return np.exp(x/t) / np.sum(np.exp(x/t))
x = np.array([0.3,0.5,0.8,0.9])
softmax(x, 0.5)

array([0.11723022, 0.17488694, 0.31866479, 0.38921805])

In [60]:
#?np.random.multinomial
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # 回传多项式分布的概率
    probas = np.random.multinomial(1, preds, 1)
    # 回傳最大index
    return np.argmax(probas)

## 文本生成

In [61]:
def random_reviews(temperature):
    generated_text =  'rest assured, iron man is an absolutely amazing movie. i won'
    generated_text  = generated_text[0:60]
    
    for i in range(600):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1
            
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print(generated_text)

In [66]:
random_reviews(0.3)

dyon man it at dome the one oof than in thiss strys dimmorad dowery for such all wal guce the geven weyner. har ghamally she to in a charmy jes in the screetne my pirstween erougerat ederramicen have shemer a hin thero she als and itsoly jes anss that is movis just this caming the finm. the man ur the iron man is ovin is this oun wast th inll ints liok of starking ot the caming is fan, and not the rillly went thim ligen thats the bag filst all the une it's roked and fig the cast of his coull well ir. the man it this rose mush poome a gionlagit frous this is als youg, i manaugely courd just thegionlagit frous this is als youg, i manaugely courd just the
